# Une courte introduction à Pyro

https://pyro.ai/

Pyro est un langage probabiliste construit sur Python et PyTorch.

Comparé aux autres langages probabilistes, Pyro a mis l'accent sur l'inférence variationelle (SVI).
PyTorch permet par ailleurs d'intégrer des réseaux de neurones complets dans les modèles probabilistes pour capturer des interactions complexes entre variables aléatoires.



In [1]:
import pyro
from pyro.distributions import Uniform, Bernoulli, Beta, Normal
from pyro.infer import SVI, Trace_ELBO, MCMC, NUTS, Predictive
from pyro.optim import Adam
import pyro.infer.autoguide as autoguide

import torch 
import torch.distributions.constraints as constraints

from tqdm import tqdm

## Exercice 1 : Inférence multiple pour une pièce biaisée.

**Objectif.** L'objectif de cet exercice est de se familiariser avec Pyro et plusieurs techniques d'inférence sur un modèle bien connu.

Les primitives Pyro sont les suivantes:

- `x = pyro.sample('x', d)` implémente la construction `sample`
- `pyro.sample('y', d, obs = y)` implémente la construction `observe`

Remarque : En Pyro, les variables aléatoires (introduites par `sample`, observées ou non) doivent être associées à un nom unique le *sample site*. En général, on utilise le nom de la variable sauf en cas de conflit.

### Modèle

**Question 1** Implémenter le modèle de la pièce biaisé en Pyro. On utilisera une boucle Python pour itérer sur le tableau de données (attention au nommage des variables aléatoires).

In [2]:
def model(x):
    z = pyro.sample('x', Uniform(0.,1.))
    for i in range(len(x)):
        pyro.sample(f"x{i}", Bernoulli(z), obs=x[i])

### SVI

Il faut maintenant définir un guide (une famille variationnelle) pour SVI.
Le guide doit échantillonner les mêmes variables que le modèle (ici `z`) sur le même domaine de définition (ici $[0, 1]$).
Le guide doit également avoir la même signature que le modèle (entrées et sorties).

Les paramètres du guide sont introduits à l'aide de la primitive `pyro.param('p', v0, constraint=c)`.
Chaque paramètre est associé à un nom unique et une valeur initiale.
Le paramètre optionnel `constraint` permet de spécifier des contraintes (cf `torch.distributions.constraints`), par exemple `constraints.positive`.


**Question 2** Implémenter un guide pour le modèle précédent.

_Note_: On pourra par exemple chercher une distribution $\mathit{Beta}(\alpha, \beta)$ où $\alpha \geq 0$ et $\beta \geq 0$ sont les paramètres de la famille variationnelle. 

In [3]:
def guide(x):
    alpha = pyro.param('alpha', torch.tensor(1.), constraint=constraints.positive)
    beta = pyro.param('beta', torch.tensor(1.), constraint=constraints.positive)
    z = pyro.sample('x', Beta(alpha,beta))

Il ne reste plus qu'à lancer l'inférence.
Pyro utilise les _optimizers_ de PyTorch comme `Adam` pour trouver la valeur des paramètres du guide.

_Note_: L'ajout de noms uniques permet à Pyro de croiser les variables aléatoires du modèle et du guide pour résoudre le problème d'optimisation (toutes les variables sont ajoutées à un environnement global, on peut donc y acceder en dehors des fonctions `model` et `guide`).
Malheureusement, on perd ainsi la portée des variables et c'est au programmeur d'éviter les conflits de noms.

In [4]:
def train(model, guide, data, n_steps = 5000):
    # optimizer
    optimizer = Adam({"lr": 0.0005, "betas": (0.90, 0.999)})
    # create svi object from model and guide to run inference
    svi = SVI(model, guide, optimizer, loss=Trace_ELBO())
    # do gradient steps
    for step in tqdm(range(n_steps), ncols=80):
        svi.step(data)

In [ ]:
data = torch.tensor([1., 1., 0., 0., 0., 0., 0., 0., 0., 0.])

train(model, guide, data, n_steps= 10000)

 16%|██████▏                               | 1612/10000 [00:18<01:27, 95.49it/s]

Une fois l'inférence terminée on peut récuperer la valeur finale des paramètres et calculer les moments de la distibution $\mathit{Beta}$ obtenue.

**Question 3.** Calculer ces moments (cf $\mathit{Beta}$ distribution) à partir des valeurs de $\alpha$ et $\beta$.

In [17]:
alpha = pyro.param("alpha").item()
beta = pyro.param("beta").item()

mean = alpha / (alpha+beta)
std = (beta / (alpha * (1. + alpha + beta))) ** (1/2) 

print(f"Mean: {mean}, Std: {std}")

Mean: 0.5011894850022871, Std: 0.5745908054667463


### Autoguide

En pratique programmer un guide peut être difficile.
Pyro offre un zoo d'autoguides qui sont synthétisés à partir du modèle.

Par exemple, le plus simple `AutoDelta` cherche une distribution de Dirac (un maximum à posteriori).

In [18]:
guide = autoguide.AutoDelta(model)
train(model, guide, data)

100%|██████████████████████████████████████| 5000/5000 [00:44<00:00, 111.96it/s]


On peut ensuite afficher l'ensemble des paramètres calculés par Pyro.
Le paramètre `AutoDelta.z` vient du guide synthétisé par `AutoDelta`.

In [21]:
for name, value in pyro.get_param_store().items():
    print(name, pyro.param(name))

alpha tensor(1.0096, grad_fn=<AddBackward0>)
beta tensor(1.0049, grad_fn=<AddBackward0>)
AutoDelta.x tensor(0.2002, grad_fn=<ClampBackward1>)


_Note_: Le `param_store` contient tous les paramètres calculés depuis le début de la session Python.
Il peut être réinitialisé avec la commande `pyro.clear_param_store()`.

_Note_: Attention l'inférence est exécutée dans un espace non-contraint ($\mathbb{R}^n$).
Les resultats qui apparaissent dans le `param_store` sont donc transformés (essayer`AutoNormal` par exemple).
On peut utiliser les fonctions Pyro `guide.median()` ou `guide.quantiles([0.25, 0.5, 0.75])` pour obtenir des résultats directement interpretables.

In [22]:
print(f"Median: {guide.median()}")

Median: {'x': tensor(0.2002)}


On peut également appeler directement le guide après l'inférence pour générer des échantillons (par défaut un objet contenant toutes les variables latentes du guide).
Pour `AutoDelta`, on renvoie toujours la même valeur (Dirac), mais d'autres guides génèrent des échantillons différents à chaque appel. 

In [23]:
for i in range(5):
    print(guide(data))

{'x': tensor(0.2002, grad_fn=<ExpandBackward0>)}
{'x': tensor(0.2002, grad_fn=<ExpandBackward0>)}
{'x': tensor(0.2002, grad_fn=<ExpandBackward0>)}
{'x': tensor(0.2002, grad_fn=<ExpandBackward0>)}
{'x': tensor(0.2002, grad_fn=<ExpandBackward0>)}


**Question 4.** Essayer d'autres autoguides pour le même modèle.

In [24]:
# TODO

In [25]:
# TODO

### MCMC

Pyro permet de tester plusieurs méthodes d'inférence sur le même modèle. 
On peut par exemple essayer NUTS, un noyau optimisé de Hamiltonian Monte Carlo pour MCMC (qui est aussi la méthode d'inférence par defaut de Stan).

In [26]:
kernel = NUTS(model, jit_compile=True, ignore_jit_warnings=True)
posterior = MCMC(kernel, num_samples=1000, warmup_steps=100)
posterior.run(data)

Sample: 100%|█| 1100/1100 [00:08, 135.03it/s, step size=1.52e+00, acc. prob=0.922


Pour récupérer les échantillons de la distribution à posteriori, on peut appeler la methode `get_samples` sur `posterior`.

In [27]:
posterior.get_samples()

{'x': tensor([0.5140, 0.4411, 0.5621, 0.5819, 0.5385, 0.1150, 0.1305, 0.0827, 0.2797,
         0.3764, 0.3749, 0.2581, 0.1514, 0.1063, 0.3619, 0.3795, 0.2652, 0.3239,
         0.0861, 0.1073, 0.2111, 0.1025, 0.0904, 0.2393, 0.2848, 0.3684, 0.1926,
         0.4246, 0.1054, 0.6552, 0.4819, 0.2640, 0.2167, 0.4041, 0.3288, 0.2984,
         0.2855, 0.3191, 0.3433, 0.3433, 0.3957, 0.2173, 0.1869, 0.1304, 0.1112,
         0.0767, 0.0917, 0.0640, 0.2155, 0.2305, 0.2379, 0.3258, 0.3258, 0.1700,
         0.2042, 0.2528, 0.3541, 0.1071, 0.3109, 0.2998, 0.2584, 0.2446, 0.0762,
         0.0644, 0.0731, 0.2302, 0.2182, 0.2372, 0.3973, 0.1507, 0.2849, 0.1164,
         0.0797, 0.2312, 0.3577, 0.3007, 0.2474, 0.2170, 0.2460, 0.2288, 0.2303,
         0.3411, 0.3411, 0.2698, 0.1132, 0.1293, 0.0914, 0.0731, 0.0423, 0.0395,
         0.0495, 0.0759, 0.1170, 0.2058, 0.2091, 0.2566, 0.1324, 0.1547, 0.1209,
         0.2814, 0.1627, 0.4418, 0.2095, 0.1796, 0.2735, 0.2544, 0.2544, 0.3857,
         0.2509, 0.4343

**Question 5.** En déduire les moments de la distribution obtenue.

In [1]:
samples = posterior.get_samples()
print(f"Mean: {samples['z'].mean()}")
print(f"Std: {samples['z'].std()}")

samples['z'].mean()=tensor(0.2470)
samples['z'].std()=tensor(0.1231)

SyntaxError: cannot assign to function call (3446353137.py, line 5)

On peut également utiliser la méthode `posterior.summary()`.

In [ ]:
posterior.summary()

_Remarque_ : Il existe également une implémentation de Pyro, NumPyro, qui utilise JAX à la place de PyTorch https://num.pyro.ai/

## Exercice 2 : Titanic

Adapté de https://www.kaggle.com/c/titanic/

**Objectif.** L'objectif de cet exercice est de prédire si un passager va survivre au naufrage du Titanic en fonction de données telles que son âge, son sexe, ou le prix de son billet.

Les données sont les suivantes.

In [ ]:
import pandas as pd

raw = pd.read_csv("titanic.csv")
df = raw.loc[:,["Survived"]]
df["Sex"] = raw["Sex"].apply(lambda s: 0 if s == "male" else 1)
df["Age"] = raw["Age"].fillna(raw["Age"].median())
df["Fare"] = raw["Fare"].fillna(raw["Fare"].median())

# Dictionnary contaning all the data as torch tensors
data = { 
    k : torch.tensor(df[k]).double()
    for k in df.columns    
}

df

**Question 1.** Proposer un modèle de régression logistique pour prédire la survie d'un passager.

_Note_ Une régression logistique est similaire à une régression linéaire pour les problèmes de classification.
Le modèle général est le suivant.

\begin{align*}
a &\sim \mathcal{N}(0, 1)\\
b &\sim \mathcal{N}(0, 1)\\
l &= a x + b\\
z &= \frac{1}{1 + e^{-l}}\\
y &\sim \mathit{Bernoulli}(z)
\end{align*}

Ici les données $x$ se décomposent en $x_{\mathrm{age}}$, $x_{\mathrm{sex}}$ et $x_{\mathrm{fare}}$.
On cherche donc une régression linéaire de la forme 

$$
l = a_{\mathrm{age}} * x_{\mathrm{age}} + a_{\mathrm{sex}} * x_{\mathrm{sex}} + a_{\mathrm{fare}} * x_{\mathrm{fare}} + b
$$

In [ ]:
def model(data):
    # TODO

**Question 2.** Executer l'inférence de votre choix sur votre modèle avec les données `data`.

In [ ]:
# TODO

In [ ]:
# TODO

**Question 3.** Exploiter les résultats obtenus pour faire des prédictions sur le même jeux de données.

_Note_: On pourra utiliser l'outil `Predictive` de Pyro. 
Étant donné un modèle `model` et un ensemble d'échantillons de la distribution a posteriori `samples`, 
`Predictive(model, samples).forward(data)` génère de nouvelles valeurs pour toutes les observations (si elles sont à `None` dans `data`).
On peut ensuite moyenner ces échantillons pour obtenir une prédiction.
Dans notre cas, on pourra utiliser `Predictive(model, samples).forward({**data, "Survived":None})` pour prédire la survie d'un passager

In [ ]:
posterior_predictive = Predictive(model, samples).forward({**data, "Survived":None})

In [ ]:
df["Prediction"] = # TODO

**Question 5.** Quelle est la précision de votre modèle sur ce jeux de données ?

In [ ]:
# TODO